In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
img = cv2.imread("./cyf.png")
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]
center = (w // 2, h // 2)


def rot(img, deg=0,zoom=1):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M_1 = cv2.getRotationMatrix2D(center, deg, zoom)
    return  cv2.warpAffine(img, M_1, (w, h))

def mask(img,r1=10,dr=None):
    if dr==None:
        dr = r1
    img_mask = np.zeros_like(img)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    img_mask = cv2.circle(img_mask ,center,r1,(255,255,255),-1)
    img_mask = cv2.circle(img_mask ,center,r1-dr,(0,0,0),-1)
    return img_mask&img

def rot_s(img,deg,r1,dr=None):
    imgtmp = mask(img,r1,dr)
    return rot(imgtmp,deg)

flag =  np.zeros_like(img)
r = 4
deg = 10
from random import randint
for i in range(r,w//2,r):
    deg += randint(0,20)
    ring = rot_s(img,deg,i,r)
    ring [:,:,1] &= 0b11111000 
    gradienter = np.zeros_like(img)
    gradienter[w // 2:,:,1] = 0b11
    gradienter=rot_s(gradienter,deg,i,r)
    ring += gradienter
    flag |= ring
    

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(flag[:,:,1]&2)
plt.subplot(1,2,2)
plt.imshow(flag)
plt.show()

plt.figure(figsize=(40,15))
for bgr in range(3):
    for i in  range(8):
        plt.subplot(3,8,8*bgr+i+1)
        plt.imshow(flag[:,:,bgr]&1<<i)
cv2.imwrite('flag.png',flag)

In [ ]:
flag = cv2.imread('flag.png')[:,:,1]
h, w = flag.shape[:2]
center = (w // 2, h // 2)
r=4
plt.figure(figsize=(20,20))
from tqdm import notebook as tqdm 

rd1 =[]
flag = cv2.imread('flag.png')[:,:,1]&1
for i in tqdm.trange(1,257):
    maxl = 0 
    rd = 0
    for deg in range(360):
        im =rot_s(flag,deg,i*r,r)
        if maxl < ( np.average(im[h//2:,:]) - np.average(im[:h//2,:]) ):
            maxl = ( np.average(im[h//2:,:]) - np.average(im[:h//2,:]) )
            rd = deg
    rd1.append(rd)


flag = cv2.imread('flag.png')
flag = cv2.cvtColor(flag, cv2.COLOR_BGR2RGB)
target = np.zeros_like(flag)
r=4
for i in tqdm.trange(1,flag.shape[0]//8):
    try:
        target += rot_s(flag,rd1[i-1],i*r,r)
    except:pass
plt.figure(figsize=(20,20))
plt.imshow(target)
plt.figure(figsize=(40,15))
for bgr in range(3):
    for i in  range(8):
        plt.subplot(3,8,8*bgr+i+1)
        plt.imshow(target[:,:,bgr]&1<<i)